# Exploring Cancer Mortality Factors
S. Han and C. Chung On

# Acknowledgements
The purpose of this project is to replicate Dr. Martin Skarzynski's capstone project, which was completed using R, using Python. The capstone project is titled "Exploratory Analysis of Factors Associated with Cancer Mortality in the National Health and Nutrition Examination Survey Dataset," which was completed in partial fulfillment of Dr. Skarzynski's MPH degree from Johns Hopkins University School of Public Health.

One major modification from Dr. Skarzynski's capstone is that in this project, 50 potential predictor variables were chosen before inputing the data into a Cox Proportional Hazard Model instead of running thousands of hazard models. 

The capstone project, datasets used, and R codes can be found in the following link: https://github.com/marskar/nhanes.

## Background:
Although cancer mortality has been declining in the U.S., cancer is still the second leading cause of death. Susceptibility to cancer can be divided into modifiable and non-modifiable risk factors. Modifiable factors include body mass index (BMI) and cigarette use while non-modifiable factors include age, sex, and race/ethnicity. Data science methods can be utilized to run cancer risk prediction models, which can help find the factors most associated with cancer mortality risk. In this study, we used Python to analyze data from the Third National Health and Nutrition Examination Survey (NHANES III) and the National Death Index (NDI) Public-Use Linked Mortality Files. The goal of our analysis was to examine these datasets to identify variables associated with cancer mortality risk. 

## Methods: 
NHANES III consisted of interview, medical examination, and laboratory data sets. This data was collected from 1988 to 1994 in the United States. NHANES III was linked with mortality data from the NDI death records by the National Center for Health Statistics. 

### Methods Step 1: 
Read NHANES III and NDI datasets into pandas dataframes: (4 in total, labeled as: mort, lab, adult, and exam)

Note: When reading in csv files as pandas dataframes, python replaces blank cells (in csv) as blank strings of variable number of spaces (in dataframe). In order for python to convert all empty cells to NaN, we had to include the function 'skipinitialspace=True'. 

In [1]:
import pandas as pd
from functools import reduce
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import statsmodels.formula.api as sm

In [2]:
mort = pd.read_csv(r'C:\Users\hans11\Desktop\python_project\mort.csv', skipinitialspace=True)
lab = pd.read_csv(r'C:\Users\hans11\Desktop\python_project\lab.csv', skipinitialspace=True)
adult = pd.read_csv(r'C:\Users\hans11\Desktop\python_project\adult.csv', skipinitialspace=True, low_memory=False)
exam = pd.read_csv(r'C:\Users\hans11\Desktop\python_project\exam.csv', skipinitialspace=True, low_memory=False)

In [3]:
mort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33994 entries, 0 to 33993
Data columns (total 14 columns):
SEQN            33994 non-null int64
ELIGSTAT        33994 non-null object
MORTSTAT        33994 non-null object
CAUSEAVL        33994 non-null object
UCOD_LEADING    33994 non-null object
DIABETES        33994 non-null object
HYPERTEN        33994 non-null object
PERMTH_INT      19592 non-null float64
PERMTH_EXM      17738 non-null float64
MORTSRCE_NDI    6596 non-null object
MORTSRCE_CMS    3102 non-null object
MORTSRCE_SSA    5497 non-null object
MORTSRCE_DC     2492 non-null object
MORTSRCE_DCL    0 non-null float64
dtypes: float64(3), int64(1), object(10)
memory usage: 3.6+ MB


### Methods Step 2: 
Clean NDI dataset (mort): 
    1) Only keep variables that are needed for study (i.e., SEQN, UCOD_LEADING, PERMTH_INT)
            * SEQN = unique identifier 
            * UCOD_LEADING = underlying cause of death recode from UCOD_113 (Leading causes)
            * PERMTH_INT = person months of follow-up from interview date 
    2) Remove entries with missing UCOD_LEADING or PERMTH_INT variables
    3) Create cancer mortality variable (cancer_mort) based on whether 'malignant neoplasms (COO-C97)'
    was recorded as cause of death (UCOD_LEADING) 
    4) Remove UCOD_LEADING from clean_mort dataframe (dataframe should not only have numerical data types) 

In [9]:
#1

mort_clean = mort[['SEQN', 'UCOD_LEADING', 'PERMTH_INT']]
mort_clean.info()

#33,994 entries; 3 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33994 entries, 0 to 33993
Data columns (total 3 columns):
SEQN            33994 non-null int64
UCOD_LEADING    33994 non-null object
PERMTH_INT      19592 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 796.8+ KB


In [11]:
#2: 

mort_clean = mort_clean[(mort_clean.UCOD_LEADING.notnull() & mort_clean.PERMTH_INT.notnull())]
mort_clean.info()

# 19,592 entries; 3 columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19592 entries, 0 to 33990
Data columns (total 3 columns):
SEQN            19592 non-null int64
UCOD_LEADING    19592 non-null object
PERMTH_INT      19592 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 612.2+ KB


In [12]:
#3: create cancer_mort variable

def cancer_mort(mort_clean): 
    if mort_clean['UCOD_LEADING'] == 'Malignant neoplasm (COO-C97)':
        return 1
    else:
        return 0
mort_clean['cancer_mort'] = mort_clean.apply(cancer_mort, axis=1)

mort_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19592 entries, 0 to 33990
Data columns (total 4 columns):
SEQN            19592 non-null int64
UCOD_LEADING    19592 non-null object
PERMTH_INT      19592 non-null float64
cancer_mort     19592 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 765.3+ KB


In [14]:
#4: remove UCOD_LEADING
mort_clean = mort_clean.drop('UCOD_LEADING', axis=1)
mort_clean.info()

# 19,592 entries; 3 columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19592 entries, 0 to 33990
Data columns (total 3 columns):
SEQN           19592 non-null int64
PERMTH_INT     19592 non-null float64
cancer_mort    19592 non-null int64
dtypes: float64(1), int64(2)
memory usage: 612.2 KB


### Methods Step 3:
Merge 4 dataframes together (df_merged) using SEQN

In [16]:
dfs = [mort_clean, lab, adult, exam]
df_merged = reduce(lambda left,right: pd.merge(left,right,on='SEQN'), dfs)
df_merged.info()

# df_merged = 17,738 entries; 3,962 columns 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17738 entries, 0 to 17737
Columns: 3962 entries, SEQN to PFPTECH
dtypes: float64(3644), int64(303), object(15)
memory usage: 536.3+ MB


### Methods Step 4: Exclusion Criteria
The exclusion factors included participants who: 
    1) were under 18 years of age 
    2) had a self-reported cancer at baseline, and 
    3) had missing values for variables related to the study

In [17]:
# Exclusion Crtieria #1: 
    # HSAGEIR = age at interview
    
df_merged.HSAGEIR.describe()
# Age range = 18-90 

count    17738.000000
mean        47.560886
std         20.221005
min         18.000000
25%         30.000000
50%         44.000000
75%         65.000000
max         90.000000
Name: HSAGEIR, dtype: float64

In [23]:
""" Exclusion Criteria #2: 
    HAC1N = 'doctor ever told you had: skin cancer'
    HAC1O = 'doctor ever told you had: other cancer'
        1 = Yes
        2 = No
        9 = Don't know
    Only keep those who answered 2 for HAC1N or HAC1O """ 

df_merged = df_merged[(df_merged.HAC1N == 2) & (df_merged.HAC1O == 2)]
df_merged.info()
#df_merged = 16,404 entries; 3,962 columns

AttributeError: 'NoneType' object has no attribute 'HAC1N'

In [19]:
# Exclusion Criteria #3: 
    # 15 variables with object type. Check to see if they are necessary for study purposes: 

object_list = list(df_merged.select_dtypes(include=['object']).columns)
object_list

['PHPSNTI',
 'PHPDRTI',
 'PHPBEST',
 'HAJ12',
 'HAX18A',
 'HAX18B',
 'HAX18C',
 'HAZA1CC',
 'DEPSTLC1',
 'DEPSTLC2',
 'DEPSTLC3',
 'DEPSTLC4',
 'DEPSTLC5',
 'DEPSTLC6',
 'SPPTIME']

In [22]:
# all 15 variables that are object type are not necessary for study purposes
# delete these 15 variables from dataframe (df_merged)

df_merged2 = df_merged.drop(df_merged.select_dtypes(['object']), inplace=True, axis=1)

AttributeError: 'NoneType' object has no attribute 'drop'